In [241]:
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MeanShift
from sklearn.cluster import DBSCAN

For this project we are going to be clustering movies together to give better movie recomendations. 

First, we are going to load the movie datasets containing all the ratings by users, genres, movie titles, movie ids, tags by users, and the relevence of the tag to the movie by the user. 

In [ ]:
#loading in df of tag and rating
# tag_df = pd.read_csv("../data/ml-latest/tags.csv")
# rating_df = pd.read_csv("../data/ml-latest/ratings.csv")
# tag_score_df = pd.read_csv("../data/ml-latest/genome-scores.csv")
# tag_to_num_df = pd.read_csv("../data/ml-latest/genome-tags.csv")
# movies_ids_df = pd.read_csv("../data/ml-latest/movies.csv")

Here instead of using the much larger dataset, we are using the smaller datasets but that still have enough to practice/use our clustering

In [143]:
#loading in df of tag and rating
small_tag_df = pd.read_csv("../data/ml-latest-small/tags.csv")
small_rating_df = pd.read_csv("../data/ml-latest-small/ratings.csv")
small_tag_score_df = pd.read_csv("../data/ml-latest-small/genome-scores.csv")
small_tag_to_num_df = pd.read_csv("../data/ml-latest-small/genome-tags.csv")
small_movies_ids_df = pd.read_csv("../data/ml-latest-small/movies.csv")

For some exploartory data analysis, we will look in to more about the tags and ratings to try and cluster the movies. 

In [144]:
print(len(small_tag_score_df["tagId"].unique()))
print(len(small_tag_df["tag"].unique()))
print(small_tag_score_df.head())
print(small_tag_df.head())
print(small_tag_to_num_df.head())

1128
1589
   movieId  tagId  relevance
0        1      1    0.02900
1        1      2    0.02375
2        1      3    0.05425
3        1      4    0.06875
4        1      5    0.16000
   userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200
   tagId           tag
0      1           007
1      2  007 (series)
2      3  18th century
3      4         1920s
4      5         1930s


Here, there are many tags with a helpful relevance score and also a nice conversion between tag and tagId. For this, I think that I will use the tags as the basis of clustering. To make this as accurate as possible, I will have on column for each tag and then within the column there will be the relevance of the tag to the movie. I will also be using the rating as another feature to use for the clustering. For this, I will just use averaging as another feature for clustering. In total, there will be around 1128 + 1 features for each movie that I will use for clustering. 

In [145]:
# my_df = movies_ids_df[['title', 'movieId']]
# my_df["avg_rating"] = 0

# unique_tags = tag_score_df["tagId"].unique()
# for tag in unique_tags:
#     my_df[tag] = 0
# tag_score_list = tag_score_df.values.tolist()
# for tag in tag_score_list:
#     my_df.at[tag[0], tag[1]] = tag[2]x

# my_df
#use smaller dataset right here
small_my_df = small_movies_ids_df[['title', 'movieId']]
small_my_df["avg_rating"] = 0

unique_tags = small_tag_score_df["tagId"].unique()
print(len(unique_tags))
for tag in unique_tags:
    small_my_df[tag] = 0
small_my_df.set_index("movieId")
small_my_df


1128


,title,movieId,avg_rating,1,2,3,4,5,6,7,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,Toy Story (1995),1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji (1995),2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men (1995),3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale (1995),4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II (1995),5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,Black Butler: Book of the Atlantic (2017),193581,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,No Game No Life: Zero (2017),193583,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,Flint (2017),193585,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,Bungo Stray Dogs: Dead Apple (2018),193587,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [147]:
import time
tag_id_dict = {}
for row in tqdm(small_tag_score_df.itertuples()):
    if row.movieId not in tag_id_dict:
        tag_id_dict[row.movieId] = {}
    tag_id_dict[row.movieId][row.tagId] = row.relevance

# for row in tqdm(small_tag_score_df.itertuples()):
#     tag_id_dict[(row.movieId,int(row.tagId))] = row.relevance

14862528it [00:19, 781169.09it/s]


In [75]:
print(tag_id_dict[1])

{1: 0.029000000000000026, 2: 0.023749999999999997, 3: 0.05425000000000002, 4: 0.06874999999999998, 5: 0.15999999999999998, 6: 0.19525, 7: 0.07600000000000001, 8: 0.252, 9: 0.2275, 10: 0.024000000000000018, 11: 0.5870000000000001, 12: 0.09425, 13: 0.17800000000000002, 14: 0.007000000000000006, 15: 0.035250000000000004, 16: 0.2145, 17: 0.013500000000000012, 18: 0.09725, 19: 0.66825, 20: 0.25725, 21: 0.35725, 22: 0.30225, 23: 0.049750000000000016, 24: 0.013000000000000013, 25: 0.07474999999999997, 26: 0.07050000000000001, 27: 0.1305, 28: 0.10525, 29: 0.907, 30: 0.6085, 31: 0.036999999999999984, 32: 0.23675, 33: 0.30075, 34: 0.062, 35: 0.034499999999999975, 36: 0.27325, 37: 0.11775000000000002, 38: 0.007500000000000007, 39: 0.02200000000000002, 40: 0.014500000000000013, 41: 0.017749999999999988, 42: 0.03949999999999998, 43: 0.08324999999999999, 44: 0.016249999999999987, 45: 0.2255, 46: 0.2, 47: 0.008500000000000008, 48: 0.1225, 49: 0.31299999999999994, 50: 0.15875, 51: 0.25575000000000003,

In [148]:
not_tag_movies = []
import time
def apply_tags(index, id_):
    if id_ in tag_id_dict:
        return tag_id_dict[id_][index]
    else:
        not_tag_movies.append(id_)
        return 0
for row in tqdm(small_tag_to_num_df.itertuples()):
    small_my_df[row.tagId] = small_my_df.apply(lambda x : apply_tags(row.tagId, x.movieId), axis=1)
    
# small_my_df.apply(lambda x: apply_num(x), axis=1)
# small_my_df

1128it [10:28,  1.79it/s]


In [149]:
small_my_df

,title,movieId,avg_rating,1,2,3,4,5,6,7,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,Toy Story (1995),1,0,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
1,Jumanji (1995),2,0,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
2,Grumpier Old Men (1995),3,0,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
3,Waiting to Exhale (1995),4,0,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
4,Father of the Bride Part II (1995),5,0,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,Black Butler: Book of the Atlantic (2017),193581,0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9738,No Game No Life: Zero (2017),193583,0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9739,Flint (2017),193585,0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9740,Bungo Stray Dogs: Dead Apple (2018),193587,0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


Here we now have all of the movies with their corresponding tag relevence numbers assigned to them. There may still be some movies that do not have any tags, so we should remove those. We will remove those with the list we coalated earlier. 

In [154]:
# final_df = small_my_df[small_my_df["movieId"] in not_tag_movies]
final_df = small_my_df.loc[small_my_df['movieId'].apply(lambda x: x not in not_tag_movies)]
final_df

,title,movieId,avg_rating,1,2,3,4,5,6,7,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,Toy Story (1995),1,0,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
1,Jumanji (1995),2,0,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
2,Grumpier Old Men (1995),3,0,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
3,Waiting to Exhale (1995),4,0,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
4,Father of the Bride Part II (1995),5,0,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9703,"Game Over, Man! (2018)",185435,0,0.10625,0.10775,0.03025,0.05650,0.23175,0.04575,0.05175,...,0.34725,0.03125,0.03500,0.12125,0.32200,0.08200,0.03325,0.02425,0.13725,0.02625
9705,Pacific Rim: Uprising (2018),185585,0,0.04250,0.04175,0.06475,0.07625,0.08500,0.03800,0.03425,...,0.04325,0.05850,0.03050,0.02750,0.21850,0.04525,0.03550,0.02550,0.17475,0.03100
9706,Rampage (2018),186587,0,0.08950,0.09125,0.05075,0.04900,0.19475,0.04500,0.03125,...,0.06475,0.04500,0.04000,0.01975,0.22075,0.15125,0.06150,0.04300,0.20800,0.03925
9709,Deadpool 2 (2018),187593,0,0.05450,0.05250,0.07200,0.15050,0.23125,0.12775,0.09375,...,0.15400,0.06650,0.05825,0.06675,0.19575,0.13875,0.04525,0.05775,0.16050,0.04675


In [176]:
final_df = small_my_df
final_rating = small_rating_df.groupby(["movieId"]).agg({'rating': ['mean']}).reset_index()
final_df["avg_rating"] = final_rating["rating"]
final_df.dropna(inplace=True)
final_df.drop(["title"], inplace=True, axis=1)

index_to_movie = pd.Series(final_df.movieId.values,index=df.index).to_dict()
final_df.drop(["movieId"], inplace=True, axis=1)
final_df.columns = final_df.columns.astype(str)
final_df = final_df[final_df['1'] != 0]
final_df

,avg_rating,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,3.920930,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
1,3.431818,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
2,3.259615,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
3,2.357143,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
4,3.071429,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9703,2.500000,0.10625,0.10775,0.03025,0.05650,0.23175,0.04575,0.05175,0.44700,0.19875,...,0.34725,0.03125,0.03500,0.12125,0.32200,0.08200,0.03325,0.02425,0.13725,0.02625
9705,2.500000,0.04250,0.04175,0.06475,0.07625,0.08500,0.03800,0.03425,0.16850,0.02525,...,0.04325,0.05850,0.03050,0.02750,0.21850,0.04525,0.03550,0.02550,0.17475,0.03100
9706,3.500000,0.08950,0.09125,0.05075,0.04900,0.19475,0.04500,0.03125,0.17275,0.07850,...,0.06475,0.04500,0.04000,0.01975,0.22075,0.15125,0.06150,0.04300,0.20800,0.03925
9709,1.000000,0.05450,0.05250,0.07200,0.15050,0.23125,0.12775,0.09375,0.30275,0.29850,...,0.15400,0.06650,0.05825,0.06675,0.19575,0.13875,0.04525,0.05775,0.16050,0.04675


Here we have now filled all the rows with all the features and only have dropped 18 movies from not having ratings

In [231]:
#data 
df = final_df

X = np.array(df.drop("avg_rating", axis=1))
# y = np.array(df["movieId"])

,avg_rating,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,3.920930,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
1,3.431818,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
2,3.259615,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
3,2.357143,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
4,3.071429,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9703,2.500000,0.10625,0.10775,0.03025,0.05650,0.23175,0.04575,0.05175,0.44700,0.19875,...,0.34725,0.03125,0.03500,0.12125,0.32200,0.08200,0.03325,0.02425,0.13725,0.02625
9705,2.500000,0.04250,0.04175,0.06475,0.07625,0.08500,0.03800,0.03425,0.16850,0.02525,...,0.04325,0.05850,0.03050,0.02750,0.21850,0.04525,0.03550,0.02550,0.17475,0.03100
9706,3.500000,0.08950,0.09125,0.05075,0.04900,0.19475,0.04500,0.03125,0.17275,0.07850,...,0.06475,0.04500,0.04000,0.01975,0.22075,0.15125,0.06150,0.04300,0.20800,0.03925
9709,1.000000,0.05450,0.05250,0.07200,0.15050,0.23125,0.12775,0.09375,0.30275,0.29850,...,0.15400,0.06650,0.05825,0.06675,0.19575,0.13875,0.04525,0.05775,0.16050,0.04675


The reason that I chose Affinity Propagation is because of a couple of reasons. The first reason is that unlike other models, it does not need a specification on how many clusters there are. In this case we do not know how many clusters there are. In that case, we can not say how many clusters there will be. Affinity Propagation is also good with uneven cluster sizes which you can imagine in this situation with few obscure movies that all make sense together but then there would be a much larer cluster with more popular movies populating the cluster. Also, unlike other models, you do not have to specify the distance between clusters or the distance of how large the cluster should be, these are all unknowns and would therefore have to be tampered with for a long time before achieving an answer, which may not even be most optimal given the model that you are using. 

In [233]:
clustering = AffinityPropagation().fit(X)

/usr/local/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:146: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 0.25 which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(("'random_state' has been introduced in 0.23. "


In [260]:
df = pd.DataFrame()
df["affin"] = clustering.labels_
print(len(df["affin"].unique()))
df.head()

415


,affin
0,0
1,96
2,27
3,71
4,351


Now we have clustering for all of our movies! We can now see how well afinity propagation did on putting our movies together. So that we don't have to write everything out each time, we should probably write up some functions to trasition from index to movie id to then name of the movie

In [265]:
id_to_name = pd.Series(small_movies_ids_df.title.values, index=small_movies_ids_df.movieId).to_dict()
id_to_index = {v: k for k, v in index_to_movie.items()}
name_to_id = {v: k for k, v in id_to_name.items()}

def idToMovieName(movieId):
    movie_name = id_to_name[movieId]
    return movie_name
    
def indexToMovieId(index):
    movie_id = index_to_movie[index]
    return movie_id
    
def indexToMovieName(index):
    movieId = indexToMovieId(index)
    name = idToMovieName(movieId)
    return name

def movieIdToIndex(id_):
    index = id_to_index[id_]
    return index

def movieNameToIndex(name):
    id_ = name_to_id[name]
    index = id_to_index[id_]
    return index

def findAlikeMovies(movieId):
    index = id_to_index[movieId]
    cluster_num = clustering.labels_[index]
    related_movs = []
    for i in range(len(clustering.labels_)):
        if clustering.labels_[i] == cluster_num:
            related_movs.append(i)
    title_movs = [indexToMovieName(x) for x in related_movs if x != index]
    return title_movs
    

Here we can just remind ourselves of what movies there are in our dataset

In [272]:
small_movies_ids_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [287]:
print(movieNameToIndex("Winnie Pooh (1969)"))
print(indexToMovieId(1))
findAlikeMovies(1)

9379
2


['Morning After, The (1986)',
 'Come See the Paradise (1990)',
 'Not Without My Daughter (1991)',
 'Wiz, The (1978)',
 'To End All Wars (2001)',
 'Peaceful Warrior (2006)',
 'Nines, The (2007)',
 'Zone, The (La Zona) (2007)',
 'Roots (1977)',
 'Front of the Class (2008)',
 'Thor: The Dark World (2013)']